# Address scorer
Algorithm to determine whether an address is a potential inside/smart whale

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keys 
import requests
from datetime import datetime, timedelta
import time

## Get latest block

In [9]:
def get_latest_block(t=None):
    """gets latest block as an int"""
    if t is None: t = int(time.time())
    request = f"https://api.etherscan.io/api\
                ?module=block\
                &action=getblocknobytime\
                &timestamp={t}\
                &closest=before\
                &apikey={keys.key('etherscan')}".replace(" ", "")
    res = requests.get(request)
    return int(res.json()['result'])

## Panning

In [36]:
# address = "0x6c8AdA12895975F541F483Bc155fECFda231aE34"
address = "0x950AebFD80ca6843b5407fe6472f4B7Ca1375a6F"

In [11]:
def get_etherscan(address, tx_type, start=0, end=99999999, size=10000):
    """
    returns the etherscan transactions
    tx_type:
        - "txlist" for normal eth transactions
        - "txlistinternal" for internal transactions
        - "tokentx" for erc20 transactions
        - "tokennfttx" for nft transactions
    """
    page = 1
    etherscan = []
    while True:
        request = f"https://api.etherscan.io/api?module=account\
                    &action={tx_type}\
                    &address={address}\
                    &startblock={start}\
                    &endblock={end}\
                    &page={page}\
                    &offset={size}\
                    &sort=asc\
                    &apikey={keys.key('etherscan')}".replace(" ", "")
        res = requests.get(request).json()["result"]
        etherscan += res
        if len(res)<size: break
        page += 1
    return etherscan

In [39]:
def create_transaction(nor, tok, address):
    """take in two transactions, one normal on token, that happen on same block number"""
    address = address.lower()
    side = "unknown"
    if nor is None: 
        side = "transaction"
        if address in tok["to"]: 
            side = "in"
        elif address in tok["from"]:
            side = "out"
    elif address in tok["to"] and address in nor["from"]:
        side = "buy"
    elif address in tok["from"] and address in nor["to"]:
        side = "sell"
    amt = tok["value"]
    symbol = tok["tokenSymbol"]
    block = tok["blockNumber"]
    ts = tok["timeStamp"]
    eth = nor["value"] if nor else 0
    return _create_transaction(block, ts, symbol, side, amt, eth, tok["hash"])
def _create_transaction(block, time, symbol, side, amt, eth, tx):
    return {"block":int(block),
            "time":int(time),
            "symbol":symbol,
            "side":side,
            "amt":int(amt),
            "eth":int(eth),
            "tx":tx}

In [25]:
def merge_etherscan(transactions):
    """merges the transactions with same hashes"""
    new = []
    prev_hash = ""
    prev = None
    for t in transactions:
        if t["hash"] != prev_hash:
            new.append(prev)
            prev_hash = t["hash"]
            prev = t.copy()
            prev["from"] = [prev["from"]]
            prev["to"] = [prev["to"]]
            prev["value"] = int(prev["value"])
        else:
            if t["from"] not in prev["from"]: prev["from"].append(t["from"])
            if t["to"] not in prev["to"]: prev["to"].append(t["to"])
            prev["value"] += int(t["value"])
    new.append(prev)
    return new[1:]

In [30]:
def get_transactions(address):
    """returns transactions"""
    normal = merge_etherscan(get_etherscan(address, "txlist"))
    internal = merge_etherscan(get_etherscan(address, "txlistinternal"))
    token = merge_etherscan(get_etherscan(address, "tokentx"))
    # global normal
    # global internal
    # global token
    itb = list(map(lambda x: int(x["blockNumber"]), internal))
    nob = list(map(lambda x: int(x["blockNumber"]), normal))    
    transactions = []
    i = j = 0
    for t in token:
        tbk = int(t["blockNumber"])
        # print(f"====starting block number {tbk} ======")
        tlen = len(transactions)
        while i<len(normal)-1 and (nob[i] < tbk):
            i += 1
        while j<len(internal)-1 and (itb[j] < tbk):
            j += 1
        i_ = i
        j_ = j
        # print(f"set i_ to {i_} and j_ to {j_}")
        while i_<len(normal) and nob[i_] == tbk:
            # print(f"While loop for nob")
            if normal[i_]["hash"]==t["hash"] and normal[i_]["value"] != 0:
                transactions.append(create_transaction(normal[i_], t, address))
            i_ += 1

        # print(f"j_<len(internal): {j_<len(internal)}, itb[j_] == tbk: {itb[j_] == tbk}")
        while j_<len(internal) and itb[j_] == tbk:
            # print(f"While loop for itb")
            if internal[j_]["hash"] == t["hash"] and internal[j_]["value"] != 0:
                transactions.append(create_transaction(internal[j_], t, address))
            j_ += 1
        if len(transactions) == tlen: transactions.append(create_transaction(None, t, address))
    return transactions
        

## TODO:
Next: iterate through transactions and come up with a score or just separate metrics. Considering keeping track of average buy price with respect to buy sizes and keeping track of number of wins and losses, size of mean/median win, max win/loss, number of trades etc. make sure easy to add new metrics to keep track of. 

Next: Create program to take in something like https://etherscan.io/dex?q=0xa71d0588eaf47f12b13cf8ec750430d21df04974#transactions and search through all of that

## Links
- https://etherscan.io/address/0x950aebfd80ca6843b5407fe6472f4b7ca1375a6f#tokentxns
- `0xdbc6dbf365d5124b1aa309a7dbc66fe8906c36235a30beb3f6140c07bef806f6`
- https://etherscan.io/dex?q=0xa71d0588eaf47f12b13cf8ec750430d21df04974#transactions

In [91]:
address = "0xb28ebe0ed76e01825483a723d21a6ab29743cddc"

In [92]:
t = get_transactions(address)

In [59]:
def parse_transactions(transactions):
    inventory = {}
    for t in transactions:
        s = t["symbol"]
        if t["side"] == "buy":
            if s not in inventory: inventory[s] = {"amt":0, "eth":0, "b":0, "s":0, "ps":0}
            inventory[s]["amt"] += t["amt"]
            inventory[s]["eth"] += t["eth"]
            inventory[s]["b"] += 1
        if t["side"] == "sell":
            if s not in inventory: inventory[s] = {"amt":0, "eth":0, "b":0, "s":0, "ps":0}
            if inventory[s]["eth"] and t["amt"]/t["eth"]<inventory[s]["amt"]/inventory[s]["eth"]:
                inventory[s]["ps"]+=1
            inventory[s]["s"] += 1
            inventory[s]["amt"] -= t["amt"] if inventory[s]["amt"]>t["amt"] else inventory[s]["amt"]
            inventory[s]["eth"] -= t["eth"] if inventory[s]["eth"]>t["eth"] else inventory[s]["eth"]
    return inventory

In [93]:
inv = parse_transactions(t)

In [65]:
def parse_track_record(inv):
    winrate = [0, 0]
    for i in inv:
        winrate[0]+=inv[i]["ps"]
        winrate[1]+=inv[i]["s"]
    return {"rate": winrate[0]/winrate[1], "trades": winrate[1]}

In [94]:
parse_track_record(inv)

ZeroDivisionError: division by zero